In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [33]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root = '../dataset/mnist/',
                              train=True,
                              download = True,
                              transform = transform)
train_loader = DataLoader(train_dataset,
                         shuffle = True,
                         batch_size = batch_size)
test_dataset = datasets.MNIST(root = '../dataset/mnist/',
                             train = False,
                             download = True,
                             transform = transform)
test_loader = DataLoader(test_dataset,
                        shuffle = False,
                        batch_size = batch_size)

In [34]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size = 5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320,10)
    
    def forward(self, x):
        
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x
model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [36]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        #forward
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch +1, batch_idx +1, running_loss / 300))
            running_loss = 0.0

In [37]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim = 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        print('Accuracy on test set: %d %%' % (100 * correct / total))

In [38]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.659
[1,   600] loss: 0.209
[1,   900] loss: 0.151
Accuracy on test set: 96 %
[2,   300] loss: 0.123
[2,   600] loss: 0.105
[2,   900] loss: 0.096
Accuracy on test set: 97 %
[3,   300] loss: 0.084
[3,   600] loss: 0.083
[3,   900] loss: 0.072
Accuracy on test set: 98 %
[4,   300] loss: 0.071
[4,   600] loss: 0.068
[4,   900] loss: 0.063
Accuracy on test set: 98 %
[5,   300] loss: 0.054
[5,   600] loss: 0.063
[5,   900] loss: 0.059
Accuracy on test set: 98 %
[6,   300] loss: 0.052
[6,   600] loss: 0.055
[6,   900] loss: 0.050
Accuracy on test set: 98 %
[7,   300] loss: 0.048
[7,   600] loss: 0.048
[7,   900] loss: 0.048
Accuracy on test set: 98 %
[8,   300] loss: 0.047
[8,   600] loss: 0.044
[8,   900] loss: 0.041
Accuracy on test set: 98 %
[9,   300] loss: 0.038
[9,   600] loss: 0.039
[9,   900] loss: 0.045
Accuracy on test set: 98 %
[10,   300] loss: 0.038
[10,   600] loss: 0.035
[10,   900] loss: 0.038
Accuracy on test set: 98 %
